In [1]:
# -*- coding: utf-8 -*-
# 埼玉千葉神奈川記事反映本番用
import requests
import json
from wordpress_xmlrpc.methods import media
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media, posts
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc import Client, WordPressPost
from PIL import Image, ImageDraw, ImageFont
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import requests
import urllib.request as req
import glob
import json
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import timedelta
import datetime
import os
from dotenv import load_dotenv
from selenium.webdriver.common.by import By
from wordpress_xmlrpc import Client, WordPressPost, methods
from src.utils import *
print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()






読み込み完了
ライブラリの読み込み完了


True

In [ ]:
 
post.post_status = 'private'
 
wp.call(posts.EditPost(記事ID, post))
 
#ただしこのままでは記事タイトルが消され、nontitleになってしまします。（バグ？）
#よって一度記事タイトルを取得し、記事タイトルと共に編集内容を代入します
 

result = wp.call(posts.GetPost(post_id))
 
post = WordPressPost()
 
post.title = result.title #記事タイトル
post.post_status = "publish" #公開状態
 
wp.call(posts.EditPost(post_id, post))

In [35]:
post.content = 'テスト２'

In [27]:
for post in posts :
    vars(post)
    break

In [ ]:
get_content_id(self)
    posts = wp.call(methods.posts.GetPosts({"number": 100 "offset": 0}))

In [13]:
# 20件ずつ取得したい場合、
number = 20  # 一度に取得するコンテンツ数
offset = 0  # オフセット
while True:
    posts = wp.call(methods.posts.GetPosts({"number": number, "offset": offset}))
    if len(posts) == 0:
        break  # これ以上取得するコンテンツがない場合

    for post in posts:
        print(post.id, post.title)

    offset += number
    if offset > 10:
        break

NameError: name 'wp' is not defined

In [2]:
class Blog():
    def __init__(self):
        WORDPRESS_ID = os.environ['WORDPRESS_ID']
        WORDPRESS_PW = os.environ['WORDPRESS_PW']
        WORDPRESS_URL = os.environ['WORDPRESS_URL']
        self.wp = Client(WORDPRESS_URL, WORDPRESS_ID, WORDPRESS_PW)
        
    def add_target_date (self, target_day_number:int):
        week_list = [ '(日)','(月)', '(火)', '(水)', '(木)', '(金)', '(土)','(日)']
        target_date:datetime = datetime.date.today() + datetime.timedelta(days=target_day_number)
        #datetime(2023,3,3)
        self.target_date:datetime = datetime.date.today() + datetime.timedelta(days=target_day_number)
        #3月3日(火)
        self.target_date_string_jp:str = target_date.strftime('%m').lstrip('0') + '月' + target_date.strftime('%d').lstrip('0') + '日'  +week_list[target_date.isoweekday()]
        #2023-03-03
        self.target_date_string_sql:str = target_date.strftime('%Y-%m-%d')
        print(f'インスタンに日付:{self.target_date_string_jp}など三つの変数が追加されました')
    
    def upload_image(self,in_image_file_name, out_image_file_name):
        if os.path.exists(in_image_file_name):
            with open(in_image_file_name, 'rb') as f:
                binary = f.read()

            data = {
                "name": out_image_file_name,
                "type": 'image/png',
                "overwrite": True,
                "bits": binary
            }

            media_id = self.wp.call(media.UploadFile(data))['id']
            print(in_image_file_name.split('/')
                [-1], 'Upload Success : id=%s' % media_id)
            return media_id
        else:
            print(in_image_file_name.split('/')[-1], 'NO IMAGE!!')

    def post_line(self,message):
        url = "https://notify-api.line.me/api/notify"
        token = os.environ['LINE_TOKEN']
        headers = {"Authorization" : "Bearer "+ token}
        payload = {"message" :  message}
        #imagesフォルダの中のgazo.jpg
        #print('image_path',image_path)
        #files = {"imageFile":open(image_path,'rb')}
        post = requests.post(url ,headers = headers ,params=payload) 


    def get_post_list(self):
        '''投稿一覧を取得する関数
        一回で100記事まで取得できる'''
        post_list:list[WordPressPost] = self.wp.call(methods.posts.GetPosts({"number": 10, "offset":0}))
        self.post_list = post_list

    def generate_thumbnail(self) :
        image_path = r'image\image_source\千葉.jpg'  # mac
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        write_image_context = f'''{self.target_date_string_jp }\n{self.prefecture}\n取材予定まとめ'''
        font_path = r"font/LightNovelPOPv2.otf"
        font = ImageFont.truetype(font_path, size=240)
        draw.multiline_text((130, 20), write_image_context, fill=(255, 255, 255),align='center',font=font, spacing=50, stroke_width=5, stroke_fill=(55, 55, 55))
        thumbnail_image_path = fr'image\thumbnail_{self.target_date}_{self.prefecture}.jpg'
        image.save(thumbnail_image_path)
        self.thumbnail_image_path = thumbnail_image_path

    def post_blog(self):
        title = f"【{self.prefecture}】{self.target_date_string_jp} パチンコスロットイベント取材まとめ"
        # Blog Content (html)
        body = f'''
        テスト_{self.prefecture}_{self.target_date_string_jp}_1
        '''
        # publish or draft
        status = 'draft' # "publish"　or# 'draft'

        # Category keyword
        cat1 = '取材予定まとめ'
        cat2 = ''
        cat3 = ''

        # Tag keyword
        tag1 = f'{self.prefecture}'
        tag2 = f'{self.target_date}'

        slug = f"pledge_report_{self.target_date}_{self.prefecture}"

        # Post
        post = WordPressPost()
        post.title = title
        post.content = body
        post.post_status = status
        post.terms_names = {
            "category": [cat1],
            "post_tag": [tag1, tag2],
        }
        post.slug = slug

        #サムネイル関連
        output_thumbnail_path = f'thumbnail_{self.target_date}_{self.prefecture}.jpg'
        media_id = self.upload_image(self.thumbnail_image_path, output_thumbnail_path)
        post.thumbnail = media_id

        #Post Time
        #post.date = datetime.datetime.now()

        self.wp.call(NewPost(post))
        print('記事書き込み完了')
        # except Exception as e:
    #     post_line(f'{tomorrow}分ブログ投稿失敗\n{e}')



In [13]:
datetime.datetime.now()

datetime.datetime(2023, 3, 25, 17, 5, 35, 752765)

【長野県】3月28日(火) パチンコスロットイベント取材まとめ 37446


In [14]:
post.content

'テスト_長野県_3月25日(土)_1'

In [15]:
post.title += '_変更'
post.content += '_変更'

'37443'

<WordPressPost: b'\xe3\x80\x90\xe9\x95\xb7\xe9\x87\x8e\xe7\x9c\x8c\xe3\x80\x913\xe6\x9c\x8825\xe6\x97\xa5(\xe5\x9c\x9f) \xe3\x83\x91\xe3\x83\x81\xe3\x83\xb3\xe3\x82\xb3\xe3\x82\xb9\xe3\x83\xad\xe3\x83\x83\xe3\x83\x88\xe3\x82\xa4\xe3\x83\x99\xe3\x83\xb3\xe3\x83\x88\xe5\x8f\x96\xe6\x9d\x90\xe3\x81\xbe\xe3\x81\xa8\xe3\x82\x81'>

In [25]:
ret = blog.wp.call(methods.posts.DeletePost(int(post.id)))
print(ret)

True


In [20]:
int(post.id)

'37443'

In [46]:
WORDPRESS_ID = os.environ['WORDPRESS_ID']
WORDPRESS_PW = os.environ['WORDPRESS_PW']
WORDPRESS_URL = os.environ['WORDPRESS_URL']
wp = Client(WORDPRESS_URL, WORDPRESS_ID, WORDPRESS_PW)

post.title += "編集したタイトル"
post.content += "編集した内容"

# コンテンツを更新する。
ret = wp.call(methods.posts.EditPost(post.id, post))
print(ret)

Fault: <Fault 404: '無効な添付ファイル ID です。'>

In [26]:
prefecture_list = ['長野県','静岡県','山梨県','愛知県']
for target_day_number  in range(3,6):
    for prefecture in prefecture_list:
        blog = Blog()
        print(prefecture,target_day_number)
        blog.add_target_date (target_day_number)
        blog.prefecture = prefecture
        blog.add_target_date(target_day_number)
        title = f"【{blog.prefecture}】{blog.target_date_string_jp } パチンコスロットイベント取材まとめ"
        print(title)
        blog.generate_thumbnail()
        blog.post_blog()
        break
    break

長野県 3
インスタンに日付:3月28日(火)など三つの変数が追加されました
インスタンに日付:3月28日(火)など三つの変数が追加されました
【長野県】3月28日(火) パチンコスロットイベント取材まとめ
image\thumbnail_2023-03-28_長野県.jpg Upload Success : id=37445
記事書き込み完了


In [29]:
blog.get_post_list()
for post in blog.post_list:
    print(post.title,post.id)
    break

【長野県】3月28日(火) パチンコスロットイベント取材まとめ 37446


In [45]:
post.id

'37446'

In [49]:
post_id:int = int(post.id)
post = blog.wp.call(methods.posts.GetPost(post_id))
post

<WordPressPost: b'\xe3\x80\x90\xe9\x95\xb7\xe9\x87\x8e\xe7\x9c\x8c\xe3\x80\x913\xe6\x9c\x8828\xe6\x97\xa5(\xe7\x81\xab) \xe3\x83\x91\xe3\x83\x81\xe3\x83\xb3\xe3\x82\xb3\xe3\x82\xb9\xe3\x83\xad\xe3\x83\x83\xe3\x83\x88\xe3\x82\xa4\xe3\x83\x99\xe3\x83\xb3\xe3\x83\x88\xe5\x8f\x96\xe6\x9d\x90\xe3\x81\xbe\xe3\x81\xa8\xe3\x82\x81'>

In [53]:
vars(methods.posts.EditPost)

mappingproxy({'__module__': 'wordpress_xmlrpc.methods.posts',
              '__doc__': '\n    Edit an existing blog post.\n\n    Parameters:\n        `post_id`: ID of the blog post to edit.\n        `content`: A :class:`WordPressPost` instance with the new values for the blog post.\n\n    Returns: `True` on successful edit.\n    ',
              'method_name': 'wp.editPost',
              'method_args': ('post_id', 'content')})

In [55]:
post.content

'テスト_長野県_3月28日(火)_1'

In [59]:
import time
from datetime import datetime
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
import locale

# 時間を日本時間にセットする
locale.setlocale(locale.LC_TIME, 'ja_JP.UTF-8')



# クライアントの呼び出しなど

post = WordPressPost()

# 記事の編集 ##########
post.post_status = "draft" #publish:投稿 or draft:下書き

# 過去の日付にするとその日に投稿されたことになる。
# 未来の日付にすると投稿が予約にされる
post.date = datetime.now()

post.title = "タイトル"
post.content = "本文（HTML形式でも良い）"

# 記事の編集おわり ##########

wp.call(NewPost(post))


'37449'

In [61]:
import time
from datetime import datetime
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost,GetPost,EditPost
import locale

In [63]:
vars(post)

{'_def': {'id': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f504c9a0>,
  'user': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f504c1f0>,
  'date': <wordpress_xmlrpc.fieldmaps.DateTimeFieldMap at 0x193d1af1f10>,
  'date_modified': <wordpress_xmlrpc.fieldmaps.DateTimeFieldMap at 0x193d1af1fa0>,
  'slug': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f504cf70>,
  'post_status': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f504c340>,
  'title': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193d1b05040>,
  'content': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f504ce20>,
  'excerpt': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f504c2e0>,
  'link': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f5042fa0>,
  'comment_status': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f5042850>,
  'ping_status': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f5042220>,
  'terms': <wordpress_xmlrpc.fieldmaps.TermsListFieldMap at 0x193d1b050a0>,
  'terms_names': <wordpress_xmlrpc.fieldmaps.FieldMap at 0x193f5042b50

In [62]:



# 編集したい記事を呼び出す
post = wp.call(GetPost(37449))

# 記事を編集する（新規投稿時と同じ）

post.title = "タイトル1"
post.content = "本文1（HTML形式でも良い）"

post.date = datetime.now()

# 記事の投稿
wp.call(EditPost(37449, post))

Fault: <Fault 404: '無効な添付ファイル ID です。'>